In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import pandas as pd
import numpy as np
import statsmodels as sm
import sklearn as skl
import sklearn.preprocessing as preprocessing
import sklearn.linear_model as linear_model
import sklearn.cross_validation as cross_validation
import sklearn.metrics as metrics
import sklearn.tree as tree
import seaborn as sns
import matplotlib.pyplot as plt
from math import *
from scipy.stats import normaltest

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

# LOAD DATA

In [ ]:
data = pd.read_csv("../input/adult.data", 
    names=["Age", "Workclass", "fnlwgt", "Education", "Education-Num", "Martial Status", "Occupation", "Relationship", "Race", "Sex", "Capital Gain", "Capital Loss", "Hours per week", "Country", "Target"],
    sep=r'\s*,\s*',
    engine='python',
    na_values="?")

print(data.tail())

# ANALYSE DATA

In [ ]:
data.info()

On observe des données manquate sont présentes dans les variables "Workclass", "Occupation" et "Country".

In [ ]:
fig = plt.figure(figsize=(20,15))
cols = 5
rows = ceil(float(data.shape[1]) / cols)
for i, column in enumerate(data.columns):
    ax = fig.add_subplot(rows, cols, i + 1)
    ax.set_title(column)
    if data.dtypes[column] == np.object:
        data[column].value_counts().plot(kind="bar", axes=ax)
    else:
        data[column].hist(axes=ax)
        plt.xticks(rotation="vertical")
plt.subplots_adjust(hspace=0.7, wspace=0.2)

# CONVERT ALL VAR QUALI TO  VAR QUANTI

In [ ]:
data.dropna()

In [ ]:
for i, column in enumerate(data[['Workclass', 'Education', 'Martial Status', 'Occupation', 'Relationship', 'Race',
                                 'Sex', 'Country', 'Target']].columns):
    l = list(set(data[column]))
    data[column] = data[column].map(lambda x: l.index(x))

In [ ]:
print(data.head())

### Matrice des correlations

In [ ]:
hmap = data.corr()
plt.subplots(figsize=(12, 9))
sns.heatmap(hmap, vmax=.8,annot=True,cmap="seismic", square=True);

# SPLIT DATASET

In [ ]:
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn import metrics


X=data.drop('Target', axis=1).values
y= data[['Target']].values

X_train, X_test, y_train, y_test = train_test_split(X ,y, test_size=0.3, random_state=21, stratify=y)

# KNN

In [ ]:
#Tuning the model
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

param_grid= {'n_neighbors': np.arange(1,80)}
knn = KNeighborsClassifier()
knn_cv=GridSearchCV(knn, param_grid, cv=5)
y = y.reshape(32561,)
knn_cv.fit(X, y)
print(knn_cv.best_params_)
print(knn_cv.best_score_)

In [ ]:
#KNN
model=KNeighborsClassifier(n_neighbors=78) 
model.fit(X_train,y_train)
prediction=model.predict(X_test)
print('The accuracy of the KNN is',metrics.accuracy_score(prediction,y_test))

In [ ]:
from xgboost import XGBClassifier

X2=df[['education.num','age','hours.per.week', 'capital.gain']].values
y2= df[['income']].values

X2_train, X2_test, y2_train, y2_test = train_test_split(X2 ,y2, test_size=0.3, random_state=21, stratify=y)

# fit model no training data
xgbc = XGBClassifier()
xgbc.fit(X2_train, y2_train)
prediction2=xgbc.predict(X2_test)
print('The accuracy of the xGB is',metrics.accuracy_score(prediction2,y2_test))

# DECISION TREE

In [ ]:
#SETTING UP DECISSION TREES

clf = tree.DecisionTreeClassifier()
clf.fit(X_train, y_train)
predn=clf.predict(X_test)
print('The accuracy of the model is',metrics.accuracy_score(predn,y_test))

# SVM

In [ ]:
#SVM
from sklearn import svm
svc = svm.SVC(kernel='linear')
svc.fit(X_train, y_train)
y_pred=svc.predict(X_test)

print("Test set predictions:\n {}".format(y_pred))
print(svc.score(X_test,y_test))